In [1]:
import numpy as np
import os
import sys
sys.path.append(r'C:\Users\Max Tost\Desktop\Notebooks\SPC Neural Network Project')
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb
import matplotlib.pyplot as plt # for plotting
import seaborn as sns # for plotting
import pandas as pd # for data manipulation
from Models.load_data import *

%load_ext autoreload
%autoreload 2

In [4]:
# Load the Iris dataset
iris = load_iris()
X = iris.data  # Features
y = iris.target  # Labels

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [2]:
path = r'C:\Users\Max Tost\Desktop\Notebooks\SPC Neural Network Project\Training_data'
features_list = os.listdir(os.path.join(path, r'features'))

# Replace 'path' and 'features_list' with your actual base path and list of feature file names.
X, y = load_all_data(path, features_list)

print("Combined features shape:", X.shape)
print("Combined targets shape:", y.shape)

Key ECE_PF not in JETno100176.csv. Filling it with zeros.
Key RNT not in JETno100184.csv. Filling it with zeros.
Key ECE_PF not in JETno100198.csv. Filling it with zeros.
Key RNT not in JETno100228.csv. Filling it with zeros.
Key ECE_PF not in JETno100535.csv. Filling it with zeros.
Key DAI_EDG7 not in JETno80609.csv. Filling it with zeros.
Key DAI_EDG7 not in JETno80848.csv. Filling it with zeros.
Key DAI_EDG7 not in JETno80853.csv. Filling it with zeros.
Key DAI_EDG7 not in JETno81187.csv. Filling it with zeros.
Key DAI_EDG7 not in JETno81568.csv. Filling it with zeros.
Key DAI_EDG7 not in JETno81586.csv. Filling it with zeros.
Key ECE_PF not in JETno81929.csv. Filling it with zeros.
Key DAI_EDG7 not in JETno82017.csv. Filling it with zeros.
Key DAI_EDG7 not in JETno82257.csv. Filling it with zeros.
Key DAI_EDG7 not in JETno82466.csv. Filling it with zeros.
Key ECE_PF not in JETno82811.csv. Filling it with zeros.
Key DAI_EDG7 not in JETno82826.csv. Filling it with zeros.
Key DAI_EDG7

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
features_sequence = ['SSXcore', 'IPLA', 'DAO_EDG7', 'RNT', 'DAI_EDG7', 'ECE_PF']

data = pd.DataFrame(X, columns=features_sequence)
data['label']=y
data['label'] = data['label'].map({0: 'No RE', 1: 'RE'})

In [ ]:
sns.pairplot(data, hue='label')
plt.show()

KeyboardInterrupt: 

Error in callback <function _draw_all_if_interactive at 0x0000020BA0E57880> (for post_execute):


In [4]:
from imblearn.over_sampling import SMOTE

# Instantiate SMOTE with a fixed random_state for reproducibility
smote = SMOTE(random_state=42)

# Apply SMOTE only on the training data
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

In [9]:
len(np.where(y_train_res==1)[0]), len(np.where(y_train_res==0)[0])

(1420571, 1420571)

In [11]:
# Convert the data into DMatrix
dtrain = xgb.DMatrix(X_train_res, label=y_train_res)
dtest = xgb.DMatrix(X_test, label=y_test)

In [12]:
# Set parameters for binary classification (with 6 features in your dataset)
params = {
    "objective": "binary:logistic",  # Binary classification using logistic regression
    "eval_metric": "logloss",        # Evaluation metric suitable for binary classification
    "eta": 0.3,                      # Learning rate
    "max_depth": 4                   # Maximum tree depth
}

In [13]:
num_rounds = 100
bst = xgb.train(params, dtrain, num_rounds)

In [15]:
# Predict on the test set
y_pred = bst.predict(dtest)

# Convert predictions to integers
y_pred = y_pred.astype(int)

In [16]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
#

Accuracy: 0.99


In [17]:
from sklearn.metrics import classification_report

# Assuming you have predictions from your model
print(classification_report(y_test, y_pred))

C:\Users\Max Tost\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Max Tost\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99    355047
         1.0       0.00      0.00      0.00      4954

    accuracy                           0.99    360001
   macro avg       0.49      0.50      0.50    360001
weighted avg       0.97      0.99      0.98    360001



C:\Users\Max Tost\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
